In [1]:
import xarray as xr
import pandas as pd
import numpy as np
import tools.config as config
import os
from tools.helper_plot import set_plot_style, draw_12_price, draw_10_price, draw_22_price


base_dir = f"../../../output/{config.TASK_NAME}/carbon_price/0_base_data"
input_file ='carbon_low_10'
year = 2050

def recalculate_carbon_price(df):
    scenario_list = [
        'carbon_low_bio_10', 'carbon_low_bio_20', 'carbon_low_bio_30',
        'carbon_low_bio_40', 'carbon_low_bio_50',
        'carbon_high_bio_10', 'carbon_high_bio_20', 'carbon_high_bio_30',
        'carbon_high_bio_40', 'carbon_high_bio_50'
    ]

    for sc in scenario_list:
        # 构造相关scenario名字
        # 假定Counterfactual名字规则为："Counterfactual_" + scenario
        # 比如 carbon_low_bio_10 → Counterfactual_carbon_low_bio_10
        counter_sc = f'Counterfactual_{sc}'
        # 另一个名字规则，比如 carbon_low_bio_10 → carbon_low_10
        base_sc = sc.replace('_bio_', '_')

        # 找到所有年份
        for year in df.loc[df['scenario'] == sc, 'Year']:
            # 取同年份的Counterfactual和base值
            v_counter = df.loc[(df['scenario'] == counter_sc) & (df['Year'] == year), 'data']
            v_base = df.loc[(df['scenario'] == base_sc) & (df['Year'] == year), 'data']
            # 如果二者都存在，则更新
            if not v_counter.empty and not v_base.empty:
                diff = float(v_counter.values[0]) - float(v_base.values[0])
                df.loc[(df['scenario'] == sc) & (df['Year'] == year), 'data'] = diff
            else:
                print(f"Warning: missing data for year {year} in {counter_sc} or {base_sc}.")
    return df

task_name = config.TASK_NAME
# task_name = '20250922_Paper2_Results_HPC_test'
input_dir = f'../../../output/{task_name}/carbon_price/1_draw_data'
output_dir = f"../../../output/{task_name}/carbon_price/3_Paper_figure"
carbon_price_da = xr.open_dataarray(f"{input_dir}/xr_carbon_price.nc")
bio_price_da = xr.open_dataarray(f"{input_dir}/xr_bio_price.nc")


In [2]:
df = carbon_price_da.to_dataframe().reset_index()
df.pivot(index='Year', columns='scenario', values='data')

scenario,Counterfactual_carbon_high_bio_10,Counterfactual_carbon_high_bio_20,Counterfactual_carbon_high_bio_30,Counterfactual_carbon_high_bio_40,Counterfactual_carbon_high_bio_50,Counterfactual_carbon_low_bio_10,Counterfactual_carbon_low_bio_20,Counterfactual_carbon_low_bio_30,Counterfactual_carbon_low_bio_40,Counterfactual_carbon_low_bio_50,...,carbon_low_10,carbon_low_20,carbon_low_30,carbon_low_40,carbon_low_50,carbon_low_bio_10,carbon_low_bio_20,carbon_low_bio_30,carbon_low_bio_40,carbon_low_bio_50
Year,,,,,,,,,,,,,,,,,,,,,
2011,1.645205e+04,1.642652e+04,1.647620e+04,16364.898438,1.639251e+04,1.645205e+04,1.642652e+04,1.647620e+04,16364.898438,1.639251e+04,...,1.612681e+04,1.612681e+04,1.612681e+04,1.612681e+04,1.612681e+04,4526.575684,3.639632e+03,4182.620117,5095.410156,-13717.966797
2012,-6.896334e+05,-5.884382e+05,7.362358e+06,-651019.437500,-1.012211e+06,-6.896334e+05,-5.884382e+05,7.362358e+06,-651019.437500,-1.012211e+06,...,-1.477081e+06,-1.477081e+06,-1.477081e+06,-1.477081e+06,-1.477081e+06,7404.348633,5.810611e+03,3955.238281,-1878.499146,13008.920898
2013,-2.200050e+06,-1.134511e+05,2.354854e+06,-354475.500000,-2.050575e+06,-2.200050e+06,-1.134511e+05,2.354854e+06,-354475.500000,-2.050575e+06,...,-6.730301e+05,-6.730301e+05,-6.730301e+05,-6.730301e+05,-6.730301e+05,-4320.393555,3.505723e+03,-444.977203,1067.257935,61.060970
2014,1.044889e+05,1.740678e+06,5.264953e+05,104820.437500,-3.112164e+05,1.044889e+05,1.740678e+06,5.264953e+05,104820.437500,-3.112164e+05,...,1.045515e+05,1.045515e+05,1.045515e+05,1.045515e+05,1.045515e+05,-5187.200195,1.070501e+05,106665.804688,1496.223755,108091.960938
2015,9.285984e+04,-7.900015e+05,-4.652442e+05,-91.844582,-6.985693e+02,9.285984e+04,-7.900015e+05,-4.652442e+05,-91.844604,-6.985693e+02,...,9.283848e+04,9.283848e+04,9.283848e+04,9.283848e+04,9.283848e+04,42470.675781,9.580401e+04,96048.445312,-390.165771,-1005.066467
2016,-3.590325e+00,4.712077e+01,-1.438530e+00,-435.021057,-8.745801e+02,-3.590325e+00,4.712077e+01,-1.438530e+00,-435.021088,-8.745801e+02,...,-3.589427e+00,-3.589427e+00,-3.589427e+00,-3.589427e+00,-3.589427e+00,-303.556549,3.342739e+06,-895.164185,3664.872559,1562.064697
2017,-1.163389e+03,-2.137200e+03,-9.866053e+02,1205.737793,2.097410e+03,-1.163389e+03,-2.137200e+03,-9.866053e+02,1205.737793,2.097410e+03,...,-1.164690e+03,-1.164690e+03,-1.164690e+03,-1.164690e+03,-1.164690e+03,-56.546066,-6.855130e+03,295.859344,82.260193,140.728348
2018,-8.152708e+02,-1.239242e+03,-7.736593e+02,1226.795776,1.842961e+03,-8.152708e+02,-1.239242e+03,-7.736592e+02,1226.795654,1.842961e+03,...,-8.150256e+02,-8.150256e+02,-8.150256e+02,-8.150256e+02,-8.150256e+02,5873.577148,-1.664432e+03,134.595581,68.953209,92.846420
2019,-1.390014e+04,-5.545723e+02,1.482460e+03,1873.466675,2.639823e+03,-1.390014e+04,-5.545723e+02,1.482460e+03,1873.467163,2.639823e+03,...,-1.387830e+04,-1.387830e+04,-1.387830e+04,-1.387830e+04,-1.387830e+04,-259.378937,3.300414e+02,99.672249,77.884171,114.501617


In [3]:
cost_path = os.path.join(base_dir, input_file, str(year), f"xr_total_cost_{input_file}_amortised_{year}.nc")

In [4]:
out_dir = f'../../../output/{config.TASK_NAME}/carbon_price/1_draw_data'
base = f'../../../output/{config.TASK_NAME}/carbon_price/1_draw_data'
cost_da = xr.open_dataarray(os.path.join(base, 'xr_total_cost.nc'))
carbon_da = xr.open_dataarray(os.path.join(base, 'xr_total_carbon.nc'))
bio_da = xr.open_dataarray(os.path.join(base, 'xr_total_bio.nc'))

In [5]:
scen ='carbon_low_10'

In [6]:
cost_da['type'].to_pandas()

type
Ag                                                              Ag
AgMgt                                                        AgMgt
Non-ag                                                      Non-ag
Transition(ag→ag)                                Transition(ag→ag)
Transition(ag→non-ag) amortised    Transition(ag→non-ag) amortised
dtype: object

In [7]:
cost_da.to_dataframe().reset_index().query('type == "Transition(ag→non-ag) amortised"')

,scenario,Year,type,data
4,carbon_low_10,2011,Transition(ag→non-ag) amortised,9.426748
9,carbon_low_10,2012,Transition(ag→non-ag) amortised,9.426748
14,carbon_low_10,2013,Transition(ag→non-ag) amortised,9.426748
19,carbon_low_10,2014,Transition(ag→non-ag) amortised,9.426748
24,carbon_low_10,2015,Transition(ag→non-ag) amortised,9.426748
...,...,...,...,...
5979,Counterfactual_carbon_high_bio_50,2046,Transition(ag→non-ag) amortised,27807.843750
5984,Counterfactual_carbon_high_bio_50,2047,Transition(ag→non-ag) amortised,28605.130859
5989,Counterfactual_carbon_high_bio_50,2048,Transition(ag→non-ag) amortised,29396.478516
5994,Counterfactual_carbon_high_bio_50,2049,Transition(ag→non-ag) amortised,30212.658203


In [8]:
cost_da.to_dataframe().reset_index().pivot_table(index='Year', columns='scenario', aggfunc='sum', values='data')

scenario,Counterfactual_carbon_high_bio_10,Counterfactual_carbon_high_bio_20,Counterfactual_carbon_high_bio_30,Counterfactual_carbon_high_bio_40,Counterfactual_carbon_high_bio_50,Counterfactual_carbon_low_bio_10,Counterfactual_carbon_low_bio_20,Counterfactual_carbon_low_bio_30,Counterfactual_carbon_low_bio_40,Counterfactual_carbon_low_bio_50,...,carbon_low_10,carbon_low_20,carbon_low_30,carbon_low_40,carbon_low_50,carbon_low_bio_10,carbon_low_bio_20,carbon_low_bio_30,carbon_low_bio_40,carbon_low_bio_50
Year,,,,,,,,,,,,,,,,,,,,,
2011,-96.493866,-96.525688,-96.475395,-96.628471,-96.402397,-96.493866,-96.525688,-96.475395,-96.628471,-96.402397,...,-96.597588,-96.597588,-96.597588,-96.597588,-96.597588,0.103727,0.071900,0.122197,-0.030879,0.195194
2012,11.956452,11.959049,12.068795,12.049020,11.982490,11.956452,11.959049,12.068795,12.049020,11.982490,...,12.029728,12.029728,12.029728,12.029728,12.029728,-0.073275,-0.070678,0.039067,0.019292,-0.047238
2013,11.959016,11.710392,11.999337,11.992623,12.010323,11.959016,11.710392,11.999337,11.992623,12.010323,...,12.009586,12.009586,12.009586,12.009586,12.009586,-0.050570,-0.299194,-0.010249,-0.016963,0.000736
2014,-468.069916,11.922428,11.882638,-468.069031,11.768990,-468.069916,11.922428,11.882638,-468.069031,11.768990,...,-468.086609,-468.086609,-468.086609,-468.086609,-468.086609,0.016691,480.009033,479.969238,0.017583,479.855591
2015,-424.436127,12.092472,12.168771,131.193970,976.026917,-424.436127,12.092472,12.168771,131.194000,976.026917,...,-424.463806,-424.463806,-424.463806,-424.463806,-424.463806,0.027677,436.556274,436.632568,555.657776,1400.490723
2016,-31.203819,409.783386,-2.484657,943.662659,2255.425293,-31.203819,409.783386,-2.484657,943.662781,2255.425293,...,-31.195889,-31.195889,-31.195889,-31.195889,-31.195889,-0.007929,440.979279,28.711227,974.858643,2286.621338
2017,-383.304626,305.507385,1029.589478,1507.592896,3015.129883,-383.304626,305.507385,1029.589478,1507.592896,3015.129883,...,-383.283051,-383.283051,-383.283051,-383.283051,-383.283051,-0.021586,688.790466,1412.872559,1890.875977,3398.413086
2018,-327.077332,809.225586,1321.365845,2318.645508,3486.251221,-327.077332,809.225464,1321.365723,2318.645264,3486.251221,...,-326.987396,-326.987396,-326.987396,-326.987396,-326.987396,-0.089946,1136.212891,1648.353027,2645.632812,3813.238525
2019,-345.853455,1112.626953,2381.646484,3585.097168,6182.867188,-345.853455,1112.626953,2381.646729,3585.098145,6182.867188,...,-345.864838,-345.864838,-345.864838,-345.864838,-345.864838,0.011375,1458.491699,2727.511719,3930.963135,6528.731934


In [9]:
carbon_da.to_dataframe().reset_index().pivot_table(index='Year', columns='scenario', aggfunc='sum', values='data')

scenario,Counterfactual_carbon_high_bio_10,Counterfactual_carbon_high_bio_20,Counterfactual_carbon_high_bio_30,Counterfactual_carbon_high_bio_40,Counterfactual_carbon_high_bio_50,Counterfactual_carbon_low_bio_10,Counterfactual_carbon_low_bio_20,Counterfactual_carbon_low_bio_30,Counterfactual_carbon_low_bio_40,Counterfactual_carbon_low_bio_50,...,carbon_low_10,carbon_low_20,carbon_low_30,carbon_low_40,carbon_low_50,carbon_low_bio_10,carbon_low_bio_20,carbon_low_bio_30,carbon_low_bio_40,carbon_low_bio_50
Year,,,,,,,,,,,,,,,,,,,,,
2011,-0.005865,-0.005876,-0.005855,-0.005905,-0.005881,-0.005865,-0.005876,-0.005855,-0.005905,-0.005881,...,-0.005990,-0.005990,-0.005990,-0.005990,-0.005990,2.291514e-05,0.000020,0.000029,-0.000006,-0.000014
2012,-0.000017,-0.000020,0.000002,-0.000019,-0.000012,-0.000017,-0.000020,0.000002,-0.000019,-0.000012,...,-0.000008,-0.000008,-0.000008,-0.000008,-0.000008,-9.896224e-06,-0.000012,0.000010,-0.000010,-0.000004
2013,-0.000005,-0.000103,0.000005,-0.000034,-0.000006,-0.000005,-0.000103,0.000005,-0.000034,-0.000006,...,-0.000018,-0.000018,-0.000018,-0.000018,-0.000018,1.170501e-05,-0.000085,0.000023,-0.000016,0.000012
2014,-0.004480,0.000007,0.000023,-0.004465,-0.000038,-0.004480,0.000007,0.000023,-0.004465,-0.000038,...,-0.004477,-0.004477,-0.004477,-0.004477,-0.004477,-3.217722e-06,0.004484,0.004500,0.000012,0.004439
2015,-0.004571,-0.000015,-0.000026,-1.428434,-1.397180,-0.004571,-0.000015,-0.000026,-1.428434,-1.397180,...,-0.004572,-0.004572,-0.004572,-0.004572,-0.004572,6.516748e-07,0.004557,0.004546,-1.424158,-1.393431
2016,8.691084,8.696448,1.727220,-2.169235,-2.578866,8.691084,8.696448,1.727220,-2.169235,-2.578866,...,8.691050,8.691050,8.691050,8.691050,8.691050,2.611984e-05,0.000132,-0.032074,0.266001,1.463846
2017,0.329472,-0.142948,-1.043568,1.250349,1.437549,0.329472,-0.142948,-1.043568,1.250349,1.437549,...,0.329086,0.329086,0.329086,0.329086,0.329086,3.817492e-04,-0.100478,4.775487,22.986525,24.148746
2018,0.401189,-0.653000,-1.707943,1.890001,1.891658,0.401189,-0.653000,-1.707943,1.890001,1.891658,...,0.401199,0.401199,0.401199,0.401199,0.401199,-1.531363e-05,-0.682643,12.246710,38.368523,41.070389
2019,0.024881,-2.006279,1.606550,1.913617,2.342152,0.024881,-2.006279,1.606550,1.913617,2.342152,...,0.024921,0.024921,0.024921,0.024921,0.024921,-4.385640e-05,4.419118,27.364805,50.471912,57.018688
